In [ ]:
import json
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import pickle
from itertools import combinations

import pprint
import json
import glob
from random import random, randrange
import logging
from collections import defaultdict
import sys

import subprocess
import shlex
from scipy.stats import pearsonr

from datetime import datetime, timedelta
from dateutil.parser import parse
import datetime
from collections import Counter
from scipy.signal import savgol_filter
import itertools

from collections import defaultdict

import networkx as nx
import numpy
from scipy.cluster import hierarchy
from scipy.spatial import distance
import matplotlib.pyplot as plt
from datetime import datetime

from scipy.cluster.hierarchy import dendrogram

#pip install python-louvain
import community as community_louvain
from operator import itemgetter 

import powerlaw

from tqdm import trange, tqdm

import time

In [ ]:
start_time = time.time()

## Functions

In [ ]:
# There is an out_degree_centrality algorithm implemented, 
# https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.out_degree_centrality.html#networkx.algorithms.centrality.out_degree_centrality
# but it does not take into account edge weight
# and edge weight here is important
# So we do it ourselves

def find_node_w_outdegree(node,network):
    out_degree = int(0)
    # https://networkx.org/documentation/stable/reference/classes/generated/networkx.DiGraph.neighbors.html
    # .neighbors gives you the 'successors', i.e. the PT.neighbots(n*) are all the nodes n* points to.
    for neighbor in network.neighbors(node):
        out_degree += network.edges[node,neighbor]['weight']
    return(out_degree)

def find_node_w_indegree(node,network):
    in_degree = int(0)
    # https://networkx.org/documentation/stable/reference/classes/generated/networkx.DiGraph.neighbors.html
    # .neighbors gives you the 'successors', i.e. the PT.neighbots(n*) are all the nodes n* points to.
    for neighbor in network.predecessors(node):
        in_degree += network.edges[neighbor,node]['weight']
    return(in_degree)

def count_neighbors(node,network):
    neighbors = int(0)
    for neighbor in network.neighbors(node):
        neighbors += 1
    for neighbor in network.predecessors(node):
        neighbors += 1
    return(neighbors)

### Full Network

In [ ]:
PT = nx.read_gexf('PT-pruned.gexf')

In [ ]:
find_node_w_outdegree('JoeBiden',PT)

In [ ]:
count = []
neighbors = []
retweets = []
indegree = []
for node in tqdm(PT.nodes):
    count.append(PT.nodes[node]['count'])
    retweets.append(find_node_w_outdegree(node,PT))
    neighbors.append(count_neighbors(node,PT))
    indegree.append(find_node_w_indegree(node,PT))

count = np.array(count)
retweets = np.array(retweets)
neighbors = np.array(neighbors)
indegree = np.array(indegree)

In [ ]:
# We need some basic pruning to even visualize it intelligibly
# one standard deviation because its really spread
def reject_outliers(data, m=0.25):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

In [ ]:
plt.hist(reject_outliers(indegree), 200, facecolor='red')
plt.xlabel('Number of In Degree')
plt.ylabel('In Degree')
plt.title("Author Retweet Count")
plt.show()

In [ ]:
# Indeed we can do some kind of power law analysis
# https://pypi.org/project/powerlaw/
# https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0085777&type=printable

print("First with counts:")
results = powerlaw.Fit(count)
print("alpha is: " + str(results.power_law.alpha))
print("xmin is: " + str(results.power_law.xmin))
R, p = results.distribution_compare('power_law', 'lognormal')
print("p is: " + str(p))

print("--------------------------------------------")
print("Second with retweets:")
results = powerlaw.Fit(retweets)
print("alpha is: " + str(results.power_law.alpha))
print("xmin is: " + str(results.power_law.xmin))
R, p = results.distribution_compare('power_law', 'lognormal')
print("p is: " + str(p))

print("--------------------------------------------")
print("Third with neighbors:")
results = powerlaw.Fit(neighbors)
print("alpha is: " + str(results.power_law.alpha))
print("xmin is: " + str(results.power_law.xmin))
R, p = results.distribution_compare('power_law', 'lognormal')
print("p is: " + str(p))

print("--------------------------------------------")
print("Third with indegree:")
results = powerlaw.Fit(indegree)
print("alpha is: " + str(results.power_law.alpha))
print("xmin is: " + str(results.power_law.xmin))
R, p = results.distribution_compare('power_law', 'lognormal')
print("p is: " + str(p))

In [ ]:
PT.clear()

### First Slice

In [ ]:
PT1 = nx.read_gexf('PT-Slice1.gexf')

In [ ]:
count = []
neighbors = []
retweets = []
indegree = []
for node in tqdm(PT1.nodes):
    count.append(PT1.nodes[node]['count'])
    retweets.append(find_node_w_outdegree(node,PT1))
    neighbors.append(count_neighbors(node,PT1))
    indegree.append(find_node_w_indegree(node,PT1))

count = np.array(count)
retweets = np.array(retweets)
neighbors = np.array(neighbors)
indegree = np.array(indegree)

In [ ]:
# We need some basic pruning to even visualize it intelligibly
# one standard deviation because its really spread
def reject_outliers(data, m=0.25):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

In [ ]:
plt.hist(reject_outliers(indegree), 200, facecolor='red')
plt.xlabel('Number of In Degree')
plt.ylabel('In Degree')
plt.title("Author Retweet Count")
plt.show()

In [ ]:
# Indeed we can do some kind of power law analysis
# https://pypi.org/project/powerlaw/
# https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0085777&type=printable

print("First with counts:")
results = powerlaw.Fit(count)
print("alpha is: " + str(results.power_law.alpha))
print("xmin is: " + str(results.power_law.xmin))
R, p = results.distribution_compare('power_law', 'lognormal')
print("p is: " + str(p))

print("--------------------------------------------")
print("Second with retweets:")
results = powerlaw.Fit(retweets)
print("alpha is: " + str(results.power_law.alpha))
print("xmin is: " + str(results.power_law.xmin))
R, p = results.distribution_compare('power_law', 'lognormal')
print("p is: " + str(p))

print("--------------------------------------------")
print("Third with neighbors:")
results = powerlaw.Fit(neighbors)
print("alpha is: " + str(results.power_law.alpha))
print("xmin is: " + str(results.power_law.xmin))
R, p = results.distribution_compare('power_law', 'lognormal')
print("p is: " + str(p))

print("--------------------------------------------")
print("Third with indegree:")
results = powerlaw.Fit(indegree)
print("alpha is: " + str(results.power_law.alpha))
print("xmin is: " + str(results.power_law.xmin))
R, p = results.distribution_compare('power_law', 'lognormal')
print("p is: " + str(p))

In [ ]:
PT1.clear()

### Second Slice

In [ ]:
PT2 = nx.read_gexf('PT-Slice2.gexf')

In [ ]:
count = []
neighbors = []
retweets = []
indegree = []
for node in tqdm(PT2.nodes):
    count.append(PT2.nodes[node]['count'])
    retweets.append(find_node_w_outdegree(node,PT2))
    neighbors.append(count_neighbors(node,PT2))
    indegree.append(find_node_w_indegree(node,PT2))

count = np.array(count)
retweets = np.array(retweets)
neighbors = np.array(neighbors)
indegree = np.array(indegree)

In [ ]:
# We need some basic pruning to even visualize it intelligibly
# one standard deviation because its really spread
def reject_outliers(data, m=0.25):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

In [ ]:
plt.hist(reject_outliers(indegree), 200, facecolor='red')
plt.xlabel('Number of In Degree')
plt.ylabel('In Degree')
plt.title("Author Retweet Count")
plt.show()

In [ ]:
# Indeed we can do some kind of power law analysis
# https://pypi.org/project/powerlaw/
# https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0085777&type=printable

print("First with counts:")
results = powerlaw.Fit(count)
print("alpha is: " + str(results.power_law.alpha))
print("xmin is: " + str(results.power_law.xmin))
R, p = results.distribution_compare('power_law', 'lognormal')
print("p is: " + str(p))

print("--------------------------------------------")
print("Second with retweets:")
results = powerlaw.Fit(retweets)
print("alpha is: " + str(results.power_law.alpha))
print("xmin is: " + str(results.power_law.xmin))
R, p = results.distribution_compare('power_law', 'lognormal')
print("p is: " + str(p))

print("--------------------------------------------")
print("Third with neighbors:")
results = powerlaw.Fit(neighbors)
print("alpha is: " + str(results.power_law.alpha))
print("xmin is: " + str(results.power_law.xmin))
R, p = results.distribution_compare('power_law', 'lognormal')
print("p is: " + str(p))

print("--------------------------------------------")
print("Third with indegree:")
results = powerlaw.Fit(indegree)
print("alpha is: " + str(results.power_law.alpha))
print("xmin is: " + str(results.power_law.xmin))
R, p = results.distribution_compare('power_law', 'lognormal')
print("p is: " + str(p))

In [ ]:
PT2.clear()

# By Community

For each community we want:

- Proportion of overall population
- Proportion of overall out weighted edges (i.e. of retweets)
- A power law analysis


In [ ]:
PT = nx.read_gexf('PT-pruned.gexf')

In [ ]:
# clusters we care about
clusters = [13,48,6,49,104]
labels = ["Democrats","Republicans","Unorthodox","Public Health","Antivaxxers"]

In [ ]:
def author_to_cluster(author):
    return(PT.nodes[author]['louvain'])

### Proportion of overall population

In [ ]:
tot_nodes = len(PT.nodes)
def prop_nodes_in_cluster(cluster):
    count = 0
    for node in tqdm(PT.nodes()):
        if PT.nodes[node]['louvain']==cluster:
            count += 1
    prop = count/tot_nodes
    return prop
tot_nodes

In [ ]:
proportion = 0
for i in range(5):
    proportion += prop_nodes_in_cluster(clusters[i])
    print('Proportion of population of ' + str(labels[i]) + ' is: ' + str(prop_nodes_in_cluster(clusters[i])))
print('Total proportion is: ' + str(proportion))    

### Proportion of out weighted edges (retweets)

In [ ]:
tot_out_wedges = 0
for node in tqdm(PT.nodes()):
    tot_out_wedges += find_node_w_outdegree(node,PT)


def prop_wout_edges_in_cluster(cluster):
    count = 0
    for node in tqdm(PT.nodes()):
        if PT.nodes[node]['louvain']==cluster:
            count += find_node_w_outdegree(node,PT)
    prop = count/tot_out_wedges
    return prop

In [ ]:
tot_out_wedges

In [ ]:
proportion = 0
for i in range(5):
    proportion += prop_wout_edges_in_cluster(clusters[i])
    print('Proportion of out weighted edges of ' + str(labels[i]) + ' is: ' + str(prop_wout_edges_in_cluster(clusters[i])))
    
print('Total proportion of out edges is: ' + str(proportion))    

### Proportion of IN weighted edges

In [ ]:
tot_in_wedges = 0
for node in tqdm(PT.nodes()):
    tot_in_wedges += find_node_w_indegree(node,PT)  

def prop_win_edges_in_cluster(cluster):
    count = 0
    for node in tqdm(PT.nodes()):
        if PT.nodes[node]['louvain']==cluster:
            count += find_node_w_indegree(node,PT)
    prop = count/tot_in_wedges
    return prop

tot_in_wedges

In [ ]:
proportion = 0

for i in range(5):
    proportion += prop_win_edges_in_cluster(clusters[i])
    print('Proportion of in weighted edges of ' + str(labels[i]) + ' is: ' + str(prop_win_edges_in_cluster(clusters[i])))
    

print('Total proportion of in edges is: ' + str(proportion))    